In [4]:
import face_recognition as fr
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
import glob
import numpy as np
import re
import pandas as pd



In [5]:
path = "data/CK+/"
data  = listdir(path)
ignore = ["morralla",".DS_Store"]
imgs = []
state = []

In [6]:
for item in data:
    if item not in ignore:
        imgs.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
        state.extend([item for p in listdir(f"{path}{item}")])

In [7]:
state[:5], imgs[:5]

(['surprise', 'surprise', 'surprise', 'surprise', 'surprise'],
 ['data/CK+/surprise/S052_001_00000015.png',
  'data/CK+/surprise/S050_002_00000018.png',
  'data/CK+/surprise/S086_001_00000019.png',
  'data/CK+/surprise/S037_001_00000020.png',
  'data/CK+/surprise/S034_001_00000029.png'])

In [8]:
imgs = [Image.open(p) for p in imgs]

In [9]:
imgs

[<PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5F905399D0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5F9054B7C0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB901EB0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB901DF0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB901F10>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB901F70>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB891070>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB8910D0>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB891100>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB891040>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB891190>,
 <PIL.PngImagePlugin.PngImageFile image mode=L size=640x490 at 0x7F5EAB8911F0>,
 <PIL.PngImagePlugin.PngImageFile image 

In [10]:
imgs_array = []
HEIGHT, WIDTH =200,200
for f in imgs:
    img = f.convert("RGB").resize((HEIGHT, WIDTH))
    imgs_array.append(np.array(img))

In [11]:
#imgs_array = [el/255 for el in imgs_array]


In [12]:
imgs_array = np.array(imgs_array)

In [13]:
imgs_array.shape


(920, 200, 200, 3)

In [14]:
imgs_flat = imgs_array.reshape(920,200*200*3)/255


In [40]:
imgs_flat


array([[0.41176471, 0.41176471, 0.41176471, ..., 0.20392157, 0.20392157,
        0.20392157],
       [0.44313725, 0.44313725, 0.44313725, ..., 0.6745098 , 0.6745098 ,
        0.6745098 ],
       [0.44705882, 0.44705882, 0.44705882, ..., 0.21568627, 0.21568627,
        0.21568627],
       ...,
       [1.        , 0.98823529, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.28627451, 0.24705882, 0.30588235, ..., 0.42352941, 0.42352941,
        0.41176471],
       [0.3372549 , 0.3254902 , 0.38039216, ..., 0.96862745, 0.99607843,
        0.99607843]])

In [18]:
categories = list(set(state))

In [26]:
from sklearn.preprocessing import  LabelEncoder

In [28]:
le = LabelEncoder()
le.fit(categories)

LabelEncoder()

In [33]:
le.classes_

array(['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral',
       'sadness', 'surprise'], dtype='<U9')

In [34]:
y= le.transform(state)

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs_flat,y)

In [42]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [43]:
models = {
    "KNN":KNeighborsClassifier(),
    "RF":RandomForestClassifier()
}

In [44]:
for name,m in models.items():
    m.fit(X_train,y_train)

In [49]:
path = "data/predict/"
data_pred  = listdir(path)
ignore = ["morralla",".DS_Store"]
imgs_pred = []
state_pred = []

In [50]:
for item in data_pred:
    if item not in ignore:
        imgs_pred.extend([f"{path}{item}/{p}" for p in listdir(f"{path}{item}")])
        state_pred.extend([item for p in listdir(f"{path}{item}")])
imgs_pred = [Image.open(p) for p in imgs_pred]
imgs_array_pred = []
for f in imgs_pred:
    img = f.convert("RGB").resize((HEIGHT, WIDTH))
    imgs_array_pred.append(np.array(img))
imgs_array_pred = np.array(imgs_array_pred)

In [57]:
state_pred = le.transform(state_pred)

In [54]:
imgs_pred_flat = imgs_array_pred.reshape(20,200*200*3)/255


In [55]:
y_pred = {}
for name, m in models.items():
    y_pred[name] = m.predict(imgs_pred_flat)

In [58]:
from sklearn.metrics import accuracy_score
metrics = {name:accuracy_score(state_pred,pred) for name,pred in y_pred.items()}

In [59]:
metrics

{'KNN': 0.2, 'RF': 0.2}

In [60]:
prediction.argmax(axis = -1)


NameError: name 'prediction' is not defined

In [129]:
cats= y_dummies.columns
cats = [x.replace("0_","") for x in cats]

In [130]:
cats

['anger',
 'contempt',
 'disgust',
 'fear',
 'happiness',
 'neutral',
 'sadness',
 'surprise']

In [132]:
states_model = []
for i in range(len(prediction)):
    states_model.append(cats[prediction[i].argmax()])
    print(state_pred[i],cats[prediction[i].argmax()])

neutral happiness
neutral anger
neutral neutral
neutral neutral
neutral neutral
happiness neutral
happiness happiness
happiness sadness
happiness happiness
happiness neutral
happiness anger
happiness neutral
happiness anger
happiness disgust
happiness happiness
sadness neutral
sadness disgust
sadness neutral
sadness disgust
sadness neutral


In [134]:
from sklearn.metrics import accuracy_score

In [135]:
accuracy_score(state_pred,states_model)

0.3